### **Further exercises answers**

**cfpython_examples notebook**

In [ ]:
# Importing cf-python and cfplot
import cf, cfplot as cfp

#Importing numpy
import numpy as np

# Additional packages for advance plotting with cfplot
import cartopy.crs as ccrs
import matplotlib.patches as mpatches

# Importing matplotlib to show that data read through cf-python can be plotted using matplotlib
import matplotlib.pyplot as plt

# The following line allows matplotlib plots to be shown in the notebook
%matplotlib inline

In [ ]:
# Initialise a file name variable along with its path
file = '/data/example-data/chess-met_tas_gb_1km_daily_20151201-20151231.nc'

# Read the netCDF file using cf python
my_dataset = cf.read(file)[0]

# See the netCDF file
my_dataset
# The following output is a list version of the file not a cf field. A cf field is in triangular brackets <> where as a list is in square brackets []

# Writing out the Y coordinate as an array
y = my_dataset.coord('projection_y_coordinate').array

# Writing out the X coordinate as an array
x = my_dataset.coord('projection_x_coordinate').array


In [ ]:
# Initialise a file name variable along with its path
file = '/data/example-data/chess-met_tas_gb_1km_daily_20151201-20151231.nc'

# Read the netCDF file using cf python
my_dataset = cf.read(file)[0]

# See the netCDF file
my_dataset
# The following output is a list version of the file not a cf field. A cf field is in triangular brackets <> where as a list is in square brackets []

In [ ]:
# Exercise 1: Averaging time over the whole of December 2015 
my_time = my_dataset.collapse('time: mean')

#Plotting average temperature for December 2015 in degrees Celsius
cfp.reset()
cfp.gopen()
cfp.con(my_time-273.15, lines=0, colorbar_title='air_temperature (degC)')
cfp.gclose()

In [ ]:
# Exercise 2: Averaging for the 17 December
my_17dec = my_dataset.subspace(time = cf.day(17))
my_17dec = my_17dec.collapse('area: mean').squeeze()
print(my_17dec.array)


In [ ]:
# Exercise 3: Comparing London and Edinburg 
my_edingburg = my_dataset.subspace(projection_x_coordinate=332500, projection_y_coordinate=673500)
my_london    = my_dataset.subspace(projection_x_coordinate=529500, projection_y_coordinate=179500)

# Identify the time axis 
time = my_dataset.coord('time').day.array

# Plotting line plots to compare the air temperature between London and Edingburgh
fig = plt.figure(figsize=(18,7))
ax=plt.subplot(1,1,1)
plt.plot(time, my_edingburg.squeeze().array, color='blue', label='Edinburg')
plt.plot(time, my_london.squeeze().array, color='red', label='London')
plt.ylabel('Air Temperature (Kelvin)')
plt.xticks(time)
plt.xlabel('Days in 2015 December')
plt.legend()
plt.show()


In [ ]:
# Exercise 4: Debugging exercise
# Calculating the temperature difference between 30 December and 1 December 2015 and plotting one figure with all values and one only potting the differences less than 4Kelvin
my_01dec = my_dataset.subspace(time=cf.day(1))
my_30dec = my_dataset.subspace(time=cf.day(30)).squeeze()
my_diff = my_30dec - my_01dec
new_diff = my_diff.where(cf.gt(4), cf.masked)

cfp.reset()
cfp.gopen(rows=1, columns=2)
cfp.cscale('scale1')
cfp.levs(min=-5, max=5, step=1)
cfp.gpos(1)
cfp.con(my_diff, lines=0, colorbar=False, title='All values')
cfp.gpos(2)
cfp.con(new_diff, lines=0, title='Values less than 4deg difference', colorbar=True, colorbar_title='Air Temperature Difference', colorbar_orientation='horizontal', colorbar_position=[0.12,0.22,0.8,0.02])
cfp.gclose()


In [ ]:
# Exercise 5: Debugging exercise
# Calculating and plotting mean, minimum, maximum, standard deviation of the air temperature over whole of December 2015 for each grid point
my_mean = my_dataset.collapse('time: mean').squeeze()
my_min = my_dataset.collapse('time: minimum').squeeze()
my_max = my_dataset.collapse('time: maximum').squeeze()
my_sd = my_dataset.collapse('time: sd').squeeze()

cfp.reset()
cfp.gopen(rows=2, columns=2)
cfp.gpos(1)
cfp.cscale('plasma')
cfp.con(my_mean, lines=0, colorbar=True, title='(a) Mean', colorbar_orientation='vertical', colorbar_title='')
cfp.gpos(2)
cfp.cscale('viridis')
cfp.con(my_sd, lines=0, colorbar=True, title='(b) Standard Deviation', colorbar_orientation='vertical', colorbar_title='')
cfp.levs(min=265, max=289, step=2)
cfp.gpos(3)
cfp.cscale('parula')
cfp.con(my_min, lines=0, colorbar=True, title='(c) Minimum', colorbar_orientation='vertical', colorbar_title='')
cfp.gpos(4)
cfp.cscale('parula')
cfp.con(my_max, lines=0, colorbar=True, title='(d) Maximum', colorbar_orientation='vertical', colorbar_title='')
cfp.gclose()


**modelvsobserved_examples notebook**

In [ ]:
# Initialise the data path
path = '/data/example-data/'
# Read the station information csv file using numpy
my_stations = np.genfromtxt(path+'SITES_2016.csv', delimiter=',', dtype='unicode') # Unicode reads data in string type which would need to be converted to float later on

# Read the VWC data csv file using numpy
my_vwc = np.genfromtxt(path+'COSMOS_VWC_2016.csv', delimiter=',', skip_header=1)[:,1:]
my_vwc_stations = np.genfromtxt(path+'COSMOS_VWC_2016.csv', delimiter=',', dtype='unicode')[0,1:]

# Initialise a file name along with its path
file = '/data/demo-data/out-example/demo-run-more-records_subsurface_run_records_daily.nc'

# We read in a netCDF file from the modelled output and select the soil water stress (SWS) variable
my_sws = cf.read(file).select('soil_water_stress')[0]
my_sws = my_sws*100

# Identify the time information from the first column of my_vwc
time = np.genfromtxt(path+'COSMOS_VWC_2016.csv', delimiter=',', dtype='unicode')[1:,0]


In [ ]:
# Exercise 1: Find the compare to BUNNY and STGHT to model data
# Command to find the index for station BUNNY and STGHT
bunny_idx = np.where(my_vwc_stations=='BUNNY_VWC')[0][0]
stght_idx = np.where(my_vwc_stations=='STGHT_VWC')[0][0]

bunny_vwc = my_vwc[:,bunny_idx]
stght_vwc = my_vwc[:,stght_idx]

bunny_idx = np.where(my_stations=='BUNNY')[0][0]
stght_idx = np.where(my_stations=='STGHT')[0][0]

print(my_stations[bunny_idx,:])
print(my_stations[stght_idx,:])

# For BUNNY and STGHT stations the nearest grid point on the modelled grid is X = -1.25; Y = 52.75
model_sws = my_sws.subspace(X = -1.25, Y = 52.75).squeeze().array

# Plotting the output
fig = plt.figure(figsize=(18,7))
ax=plt.subplot(1,1,1)
plt.plot(time, bunny_vwc, color='b', label='BUNNY VWC')
plt.plot(time, stght_vwc, color='b', label='STGHT VWC')
plt.plot(time, model_sws, color='r', label='Model SWS')
plt.ylabel('Soil Water (Percentage)')
plt.xticks(time[0::30])
plt.xlabel('Time (DD/MM/YYYY)')
plt.legend()
plt.show()


In [ ]:
# Exercise 2: Find the minimum, maximum and mean for VWC and SWS data. 
# Calculating the statistics
my_vwc_min  = np.nanmin(my_vwc, axis=1)
my_vwc_max  = np.nanmax(my_vwc, axis=1)
my_vwc_mean = np.nanmean(my_vwc, axis=1)

my_sws_min = my_sws.collapse('area: minimum').squeeze().array
my_sws_max = my_sws.collapse('area: maximum').squeeze().array
my_sws_mean = my_sws.collapse('area: mean').squeeze().array

# Plotting the output
fig = plt.figure(figsize=(18,7))
ax=plt.subplot(1,1,1)
plt.plot(time, my_vwc_mean, color='b', label='VWC Mean')
plt.fill_between(time, my_vwc_min, my_vwc_max, color='b', alpha=0.25, label='VWC Range')
plt.plot(time, my_sws_mean, color='r', label='SWS Mean')
plt.fill_between(time, my_sws_min, my_sws_max, color='r', alpha=0.25, label='SWS Range')
plt.ylabel('Soil Water (Percentage)')
plt.xticks(time[0::30])
plt.xlabel('Time (DD/MM/YYYY)')
plt.legend()
plt.show()


In [ ]:
# Exercise 3: Compare average of multiple stations VWC with regional SWS

# Get index for stations BUNNY, STGHT, WADDN, ALIC1, CHOBH, CARDT, ROTHD, LULLN,  MORLY, RDMER
station_index = []
station_list = ['BUNNY_VWC', 'STGHT_VWC', 'WADDN_VWC', 'ALIC1_VWC', 'CHOBH_VWC', 'CARDT_VWC', 'ROTHD_VWC', 'LULLN_VWC', 'MORLY_VWC', 'RDMER_VWC']
for i in range(len(station_list)):
    station_index.append(np.where(my_vwc_stations==station_list[i])[0][0])
    
# Average only the above 10 stations 
new_vwc = np.nanmean(my_vwc[:,station_index], axis=1)

# Average only for the region south and east of BUNNY (X = -1.25; Y = 52.75)
new_sws = my_sws.subspace(X=cf.wi(-1.25,3), Y=cf.wi(50, 52.75)).collapse('area: mean').squeeze().array

# Plotting the output
fig = plt.figure(figsize=(18,7))
ax=plt.subplot(1,1,1)
plt.plot(time, new_vwc, color='b', label='Station VWC')
plt.plot(time, new_sws, color='r', label='Modelled SWS')
plt.title('South East England')
plt.ylabel('Soil Water (Percentage)')
plt.xticks(time[0::30])
plt.xlabel('Time (DD/MM/YYYY)')
plt.legend()
plt.show()
